In [93]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np

username = "artia_readonly"
password = 'ap3ljax#!0frack'
host = "artia-dashboard-psql.postgres.database.azure.com"
port = "5432"              
database = "artia_dashboard"

url = URL.create(
    "postgresql+psycopg2",
    username=username,
    password=password,
    host=host,
    port=port,
    database=database,
)

engine = create_engine(url)

with open('coverage_data.sql', 'r') as f:
    sql_query = f.read()


pdl_status_map = {
    '1': 'Preferred',
    '2': 'Non-Preferred',
    '3': 'Non-PDL',
    '4': 'No State PDL',
    '5': 'Preferred (Not Listed)',
    '6': 'Non-Preferred (Not Listed)',
    '7': 'Blank'
}
    
df = pd.read_sql(sql_query, engine)
drug_df = df
drug_df['capsule_name'] = [product + ' ' + tag if tag != '' else product for product, tag in zip(drug_df['Product'], drug_df['Coverage Tag(s)'])]
drug_df = drug_df.drop_duplicates().reset_index(drop=True)
drug_df = drug_df[['id','ST', 'Product', 'Coverage Tag(s)', 'capsule_name', 'Class', 'PDL Status', 'PDL Status Date']]
drug_df = drug_df.sort_values('id').reset_index(drop=True)
drug_df['PDL Status'] = drug_df['PDL Status'].replace(np.nan,7)
drug_df['PDL Status'] = drug_df['PDL Status'].replace(np.inf,8)
drug_df['PDL Status'] = drug_df['PDL Status'].astype(int).astype(str)
drug_df['PDL Status'] = drug_df['PDL Status'].map(pdl_status_map)
drug_df['PDL Status Date'] = drug_df['PDL Status Date'].astype(str)
drug_df.head()

,id,ST,Product,Coverage Tag(s),capsule_name,Class,PDL Status,PDL Status Date
0,1,AK,Aimovig,,Aimovig,Antimigraine Agents Other,Preferred,2025-06-01
1,2,AL,Aimovig,,Aimovig,Antimigraine Agents Other,Preferred,2025-07-01
2,3,AR,Aimovig,,Aimovig,Antimigraine Agents Other,Preferred,2025-07-01
3,4,AZ,Aimovig,,Aimovig,Antimigraine Agents Other,Preferred,2025-04-01
4,5,CA,Aimovig,,Aimovig,Antimigraine Agents Other,Non-Preferred,2025-08-01


In [94]:
nopdl_error = drug_df[(drug_df['PDL Status'] == 'No State PDL') & (drug_df['ST'].isin(['ID', 'RI']))]
nopdl_error.to_csv('No_State_PDL_Errors.csv', index=False)

In [95]:
import pandas as pd

state = "GA"
pdl_date = '2025-09-01'

drug_df_refined = drug_df[drug_df["ST"]==state][drug_df['PDL Status Date']==pdl_date].reset_index(drop=True)
drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Cytokine and CAM Antagonists']

drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Immunomodulators Asthma']
drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Immunomodulators Atopic Dermatitis']
drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Immunomodulators SLE']
drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Immunomodulators Type 1 Diabetes']
drug_df_refined = drug_df_refined[drug_df_refined['Class']!='Ophthalmics Antiinflammatory Immunomodulator'].reset_index(drop=True)

scraper_data = pd.read_csv(f'{state}/{state}_clinical_output_data.csv')
scraper_status_map = scraper_data.set_index('capsule_name')['status'].to_dict()
drug_df_refined['Scraper Status'] = drug_df_refined['capsule_name'].map(scraper_status_map)

drug_df_refined.head()

C:\Users\hgarrison\AppData\Local\Temp\ipykernel_17476\1035515975.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  drug_df_refined = drug_df[drug_df["ST"]==state][drug_df['PDL Status Date']==pdl_date].reset_index(drop=True)


,id,ST,Product,Coverage Tag(s),capsule_name,Class,PDL Status,PDL Status Date,Scraper Status
0,11,GA,Aimovig,,Aimovig,Antimigraine Agents Other,Preferred,2025-09-01,Preferred
1,62,GA,Ajovy,,Ajovy,Antimigraine Agents Other,Preferred,2025-09-01,Preferred
2,164,GA,Qulipta,,Qulipta,Antimigraine Agents Prophylactic Treatment,Preferred,2025-09-01,Preferred
3,215,GA,Reyvow,,Reyvow,Antimigraine Agents Triptans,Non-Preferred,2025-09-01,Non-Preferred
4,266,GA,Trudhesa,,Trudhesa,Antimigraine Agents Acute Treatment,Non-Preferred (Not Listed),2025-09-01,NaN


In [106]:
pref_pref =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred')) & (drug_df_refined['Scraper Status'].eq('Preferred'))]
pref_nonpref = drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred')) & (drug_df_refined['Scraper Status'].eq('Non-Preferred'))]
pref_nonpdl =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred')) & (drug_df_refined['Scraper Status'].isna())]

nonpref_pref =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred')) & (drug_df_refined['Scraper Status'].eq('Preferred'))]
nonpref_nonpref = drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred')) & (drug_df_refined['Scraper Status'].eq('Non-Preferred'))]
nonpref_nonpdl =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred')) & (drug_df_refined['Scraper Status'].isna())]

nonprefnl_pref =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].eq('Preferred'))]
nonprefnl_nonpref = drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].eq('Non-Preferred'))]
nonprefnl_nonpdl =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].isna())]

prefnl_pref =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].eq('Preferred'))]
prefnl_nonpref = drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].eq('Non-Preferred'))]
prefnl_nonpdl =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Preferred (Not Listed)')) & (drug_df_refined['Scraper Status'].isna())]

nonpdl_pref =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-PDL')) & (drug_df_refined['Scraper Status'].eq('Preferred'))]
nonpdl_nonpref = drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-PDL')) & (drug_df_refined['Scraper Status'].eq('Non-Preferred'))]
nonpdl_nonpdl =  drug_df_refined.loc[(drug_df_refined['PDL Status'].eq('Non-PDL')) & (drug_df_refined['Scraper Status'].isna())]

In [114]:
accuracy_matrix = pd.DataFrame()
accuracy_matrix['Capsule Preferred'] = [len(pref_pref),len(pref_nonpref),len(pref_nonpdl)]
accuracy_matrix['Capsule Non-Preferred'] = [len(nonpref_pref),len(nonpref_nonpref),len(nonpref_nonpdl)]
accuracy_matrix['Capsule NP-NL'] = [len(nonprefnl_pref),len(nonprefnl_nonpref),len(nonprefnl_nonpdl)]
accuracy_matrix['Capsule P-NL'] = [len(prefnl_pref),len(prefnl_nonpref),len(prefnl_nonpdl)]
accuracy_matrix['Capsule Non-PDL'] = [len(nonpdl_pref),len(nonpdl_nonpref),len(nonpdl_nonpdl)]
accuracy_matrix.index = ['PDL Scraper P', 'PDL Scraper NP', 'PDL Scraper Non-PDL']
accuracy_matrix

,Capsule Preferred,Capsule Non-Preferred,Capsule NP-NL,Capsule P-NL,Capsule Non-PDL
PDL Scraper P,320,0,7,0,0
PDL Scraper NP,0,275,2,0,1
PDL Scraper Non-PDL,12,37,193,0,116


In [118]:
audit_log = pd.concat([pref_nonpref,pref_nonpdl,nonpref_pref,nonpref_nonpdl,nonprefnl_pref,nonprefnl_nonpref,nonpdl_pref,nonpdl_nonpref]).reset_index(drop=True)
audit_log['Notes'] = ''
audit_log

,id,ST,Product,Coverage Tag(s),capsule_name,Class,PDL Status,PDL Status Date,Scraper Status,Notes
0,9852,GA,Epinephrine (Mylan Ag Epipen And Jr) 0.15mg,Mylan's EpiPen Jr AG,Epinephrine (Mylan Ag Epipen And Jr) 0.15mg My...,Self Injectable Epinephrine,Preferred,2025-09-01,NaN,
1,9903,GA,Epinephrine (Mylan Ag Epipen And Jr) 0.3mg,Mylan's EpiPen AG,Epinephrine (Mylan Ag Epipen And Jr) 0.3mg Myl...,Self Injectable Epinephrine,Preferred,2025-09-01,NaN,
2,33159,GA,Select-Ob Tab Chew,,Select-Ob Tab Chew,Prenatal Vitamins,Preferred,2025-09-01,NaN,
3,34638,GA,Stuart One Otc,,Stuart One Otc,Prenatal Vitamins,Preferred,2025-09-01,NaN,
4,72518,GA,Relion TrueMetrix Air,,Relion TrueMetrix Air,Blood Glucose Meters,Preferred,2025-09-01,NaN,
5,72622,GA,True Metrix,,True Metrix,Blood Glucose Meters,Preferred,2025-09-01,NaN,
6,72674,GA,True Metrix Air,,True Metrix Air,Blood Glucose Meters,Preferred,2025-09-01,NaN,
7,73898,GA,True Metrix Test Strips,100 ct,True Metrix Test Strips 100 ct,Blood Glucose Test Strips,Preferred,2025-09-01,NaN,
8,73899,GA,True Metrix Test Strips,25 ct,True Metrix Test Strips 25 ct,Blood Glucose Test Strips,Preferred,2025-09-01,NaN,
9,73900,GA,True Metrix Test Strips,50 ct,True Metrix Test Strips 50 ct,Blood Glucose Test Strips,Preferred,2025-09-01,NaN,
